In [1]:
import seq2seq
from seq2seq.models import SimpleSeq2Seq

Using TensorFlow backend.


In [2]:
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Activation, Input
from keras.optimizers import RMSprop
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [3]:
PADDING = 26
def padding(res_len, x):
    return '#' * (res_len - len(x)) + x

def cutting(max_len, x): 
    # rework to cut from begin, not from the end! (however, it's mb not neccesary beacause all long words are zeros)
    return x if len(x) <= max_len else x[:max_len]

In [4]:
cnt = -1
X_text = []
y_text = []
answ_text = []
all_text = ''
with open('Data/task2_lemmas_train') as f:
    for line in f:
        cnt += 1
        if cnt == 0:
            continue
        splited = line.split(',')
        X_text.append(padding(PADDING, splited[1]))
        splited_target = splited[-1].strip().split('+')
        y_text.append(padding(PADDING, splited_target[0])[::-1]) #reverse it for rnn recalc it from the end of word
        answ_text.append(splited_target[1])
#         print(line.split(','))
#         if cnt > 20:
#             break

In [5]:
all_text = ''.join(X_text) + ''.join(y_text)

In [6]:
chars = set(all_text)

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [28]:
X = generate_from_text_list(X_text)
y = generate_from_text_list(y_text)

In [8]:
def generate_from_text_list(X_text):
    X = np.zeros((len(X_text), PADDING, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(X_text):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
    return X

In [18]:
get_cl_to_id = dict((cl, ni)for ni, cl in enumerate(set(answ_text)))

In [23]:
y_cl = np.zeros((len(y_text), 3))
for cur_id, cl in enumerate(answ_text):
    y_cl[cur_id, get_cl_to_id[cl]] = 1

In [30]:
classifier = Sequential()
classifier.add(GRU(32, input_shape=(PADDING, len(chars))))
classifier.add(Dense(3))
classifier.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
classifier.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [32]:
classifier.fit(X, y_cl, batch_size=128)

Epoch 1/10
118640/118640 [==============================] - 55s - loss: 0.2418    
Epoch 2/10
118640/118640 [==============================] - 51s - loss: 0.2038    
Epoch 3/10
118640/118640 [==============================] - 53s - loss: 0.1903    
Epoch 4/10
118640/118640 [==============================] - 55s - loss: 0.1808    
Epoch 5/10
118640/118640 [==============================] - 56s - loss: 0.1754    
Epoch 6/10
118640/118640 [==============================] - 54s - loss: 0.1705    
Epoch 7/10
118640/118640 [==============================] - 56s - loss: 0.1668    
Epoch 8/10
118640/118640 [==============================] - 53s - loss: 0.1642    
Epoch 9/10
118640/118640 [==============================] - 53s - loss: 0.1609    
Epoch 10/10
118640/118640 [==============================] - 54s - loss: 0.1591    


In [34]:
list(classifier.predict(X[0:1]))

[array([  9.98935401e-01,   2.11276652e-04,   8.53303995e-04], dtype=float32)]

In [10]:
# print(X)
# print(y)
# print(answ)

In [37]:
model = SimpleSeq2Seq(input_dim=len(chars), hidden_dim=128, output_length=PADDING, output_dim=len(chars), depth=1)
model.compile(loss='mse', optimizer='rmsprop')

In [38]:
def gen_from_matrix(M):
    s = ''
    for i in M:
        s += indices_char[np.argmax(i)]
    return s[::-1]

In [39]:
model.fit(X, y, batch_size=128, nb_epoch=10)

Epoch 1/10
118640/118640 [==============================] - 153s - loss: 0.0110   
Epoch 2/10
118640/118640 [==============================] - 154s - loss: 0.0084   
Epoch 3/10
 33024/118640 [=======>......................] - ETA: 107s - loss: 0.0079

KeyboardInterrupt: 

In [40]:
X_text[0]

'##############vergognerete'

In [43]:
gen_from_matrix(model.predict(X[0:1])[0])

'#################ogggggare'

In [48]:
cnt = 0
X_test_text = []
with open('Data/task2_lemmas_test') as f:
    next(f)
    for line in f:
        splited = line.strip().split(',')
        X_test_text.append(padding(PADDING, splited[1]))
        cnt += 1
        if cnt > 10:
            break

In [51]:
X_test = generate_from_text_list(X_test_text)

In [53]:
X_test_text[0]

'##################gettonan'

In [52]:
gen_from_matrix(model.predict(X_test[0:1])[0])

'##################tttttare'

In [29]:
print(max([len(i) for i in X]))
print(max([len(i) for i in y]))
print(max([len(i) for i in answ]))

26
26
1
